In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import trange

In [24]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)
#データの正規化
x_train = x_train / 255.0
x_test = x_test / 255.0
#データ数を取得
n_train = x_train.shape[0]
n_test = x_test.shape[0]
#ベクトル化
x_train,x_test = x_train.reshape(n_train,-1),x_test.reshape(n_test,-1)
#ノルムで標準化
x_train = x_train / np.linalg.norm(x_train, ord=2, axis=1, keepdims=True)
x_test = x_test / np.linalg.norm(x_test, ord=2, axis=1, keepdims=True)

2値分類の設定とするため、猫と犬の画像データだけを抽出してくる。

In [25]:
# 2つのクラスのみを選択する（例えば猫（3）と犬（5）を選択）
classes_to_use = [3, 5]
train_filter = np.isin(y_train, classes_to_use).flatten()
test_filter = np.isin(y_test, classes_to_use).flatten()

x_train_binary = x_train[train_filter]
y_train_binary = y_train[train_filter]
x_test_binary = x_test[test_filter]
y_test_binary = y_test[test_filter]

# ラベルを0と1に変換する（猫を0、犬を1とする）
y_train_binary = (y_train_binary == classes_to_use[1]).astype(np.int32)
y_test_binary = (y_test_binary == classes_to_use[1]).astype(np.int32)

In [32]:
print(x_train_binary[1])

[0.01990507 0.01881934 0.01755265 ... 0.00922871 0.00868585 0.01121922]


単純なロジスティック回帰. y_pred = 1/(1 + exp(1 + w.x)) ただしwはパラメータ、xは学習データのベクトル,ピリオドはこの2つの内積を示す。

In [35]:
#ロジスティック回帰による二値分類
class MyClassifier:
    def __init__(self, learning_rate=0.01, epochs=100,batch_size=1024):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.weights = None
        self.bias = None

    def preprocess(self, data):
        # 入力データをフラット化する
        return data.reshape(data.shape[0], -1)

    def sigmoid(self, z):
        # シグモイド関数
        return 1 / (1 + np.exp(-z))

    # 損失関数
    def loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    def train(self, X, y):
        # データの前処理
        X = self.preprocess(X)
        num_samples, num_features = X.shape

        # 重みとバイアスの初期化
        self.weights = np.zeros(num_features)
        self.bias = 0

        # イテレーション数の計算（全データを1回見渡すためのループ数）
        iterate = num_samples // self.batch_size
        print("iteration #:",iterate)
        if num_samples % self.batch_size != 0:
            iterate += 1  # 端数を含む場合、追加で1イテレート

        # 各エポックごとに、データをシャッフルしてバッチ学習
        for epoch in trange(self.epochs, desc="current epoch"):
            # データをシャッフル
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]
            for iter in trange(iterate,desc="current iter",leave=False):
                # バッチごとに学習
                start =  iter * self.batch_size
                end = min(start + self.batch_size, num_samples)
                #学習データをバッチサイズだけsliceしてくる。
                X_batch = X[start:end]
                #正解ラベルは元のデータからして縦ベクトル([num_samples,1]の形)なので、スライス後にsqueeze()で1次元にしないとy_predの計算がおかしい感じになる
                y_batch = y[start:end].squeeze()

                # バッチ全体での予測
                # 線形モデルなのでz = w^T*x + bで計算
                z = np.dot(X_batch, self.weights) + self.bias
                # zをシグモイド関数に通すことで、予測値が0-1の範囲に収まる
                y_pred = self.sigmoid(z)

                # 勾配の計算
                error = y_pred - y_batch
                # パラメータの更新式は損失関数の微分から導かれる
                # GéronテキストのLogistic Regressionの節を参照(σ(θ^T*xi)がy_pred,yiがy_batch,xiがX_batchに対応)
                dw = np.dot(X_batch.T, error) / len(y_batch)
                db = np.sum(error) / len(y_batch)

                # パラメータの更新
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            #エポックごとに損失を表示
            if(epoch % (self.epochs/10)== 0):
                print(f'loss: {self.loss(y_pred,y_batch)} \t')

    def predict(self, X):
        # データの前処理
        X = self.preprocess(X)

        # 線形モデルの予測
        z = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(z)

        # クラスの割り当て（0 または 1）
        return (y_pred >= 0.5).astype(int)

In [47]:
# MyClassifierのインポートと初期化
classifier = MyClassifier(learning_rate=0.1, epochs = 1000, batch_size=100)

# トレーニング
classifier.train(x_train_binary, y_train_binary)

iteration #: 100


current epoch:   0%|          | 1/1000 [00:00<03:14,  5.15it/s]

loss: 0.6952640023811486 	


current epoch:  10%|█         | 100/1000 [00:06<00:55, 16.15it/s]

loss: 0.6507083717831514 	


...

current epoch:  80%|████████  | 800/1000 [00:52<00:12, 16.08it/s]

loss: 0.6521184363002067 	


current epoch:  90%|█████████ | 900/1000 [00:58<00:06, 15.96it/s]

loss: 0.628837359650653 	


current epoch: 100%|██████████| 1000/1000 [01:05<00:00, 15.33it/s]


In [48]:
# モデルの評価
y_pred = classifier.predict(x_test_binary)

# 精度の計算
accuracy = np.mean(y_pred == y_test_binary.flatten())
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.61


こんどは単純な内積w.xじゃなくて、RBFを使って10000次元のベクトル$\phi (x_i) = K(x_i,x_j)$に変換して、これに対してlogit損失関数で学習してみる。
なぜ10000次元かというと、犬or猫の2値分類と化した問題でデータ数が10000個あるので、ある特定のデータが他のデータとどれくらい似てる?という組み合わせが10000通りあるから。

In [54]:
#ロジスティック回帰、RBFカーネルを導入した二値分類
class MyClassifier_RBF:
    def __init__(self, learning_rate=0.01, batch_size=32, gamma=0.1):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.gamma = gamma
        self.alpha = None  # RBFカーネルの重み
        self.bias = 0
        self.X_train = None  # トレーニングデータ全体を保持

    def preprocess(self, data):
        # 入力データをフラット化する
        return data.reshape(data.shape[0], -1)

    def sigmoid(self, z):
        # シグモイド関数
        return 1 / (1 + np.exp(-z))

    def rbf_kernel(self, xi, xj):
        # RBFカーネルの計算
        return np.exp(-self.gamma * np.linalg.norm(xi - xj) ** 2)

    def train(self, X, y):
        # データの前処理
        X = self.preprocess(X)
        self.X_train = X
        self.y_train = y.flatten()

        num_samples = X.shape[0]

        # 重みはトレーニングデータの各データポイントに対応（αを全データ数分用意）
        self.alpha = np.zeros(num_samples)
        self.bias = 0

        # エポック数の計算（全データを1回見渡すための最小エポック数）
        # 今度はすごく時間がかかることが予想されたので、データ全体をなめるのは1回だけになるようエポック数を設定した。
        # 例: 10000データの場合、バッチサイズが100なら100エポックで全データをなめることになる
        num_batches_per_epoch = num_samples // self.batch_size
        if num_samples % self.batch_size != 0:
            num_batches_per_epoch += 1  # 端数を含む場合、追加で1バッチ

        # エポックごとに学習を実行
        for epoch in trange(num_batches_per_epoch, desc="Training Progress"):
            # データをシャッフル
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            # バッチごとに学習
            start = (epoch * self.batch_size) % num_samples
            end = min(start + self.batch_size, num_samples)
            X_batch = X[start:end]
            y_batch = y[start:end]

            # 各バッチ内の各データポイントについて学習
            for i in range(len(X_batch)):
                xi = X_batch[i]
                yi = y_batch[i]

                # RBFカーネルによる線形和の計算
                z = 0
                #他の10000データに対して類似度をRBFカーネルで計算し、重み係数αとの内積を取る
                for j in range(num_samples):
                    k_ij = self.rbf_kernel(xi, self.X_train[j])
                    z += self.alpha[j] * k_ij
                z += self.bias

                # シグモイド関数を使って予測値を計算
                y_pred = self.sigmoid(z)

                # 勾配の計算
                error = y_pred - yi
                for j in range(num_samples):
                    k_ij = self.rbf_kernel(xi, self.X_train[j])
                    self.alpha[j] -= self.learning_rate * error * k_ij
                self.bias -= self.learning_rate * error

    def predict(self, X):
        # データの前処理
        X = self.preprocess(X)
        num_samples = self.X_train.shape[0]
        y_pred = []

        # 各データポイントについて予測
        for xi in X:
            # RBFカーネルによる線形和の計算
            z = 0
            for j in range(num_samples):
                k_ij = self.rbf_kernel(xi, self.X_train[j])
                z += self.alpha[j] * k_ij
            z += self.bias

            # シグモイド関数を使って予測値を計算
            prob = self.sigmoid(z)
            y_pred.append(1 if prob >= 0.5 else 0)

        return np.array(y_pred)

In [55]:
# MyClassifierのインポートと初期化
classifier = MyClassifier_RBF(learning_rate=0.01, batch_size=32)

# トレーニング
classifier.train(x_train_binary, y_train_binary)

Training Progress:   0%|          | 0/313 [00:00<?, ?it/s]/var/folders/ts/jr_9pm0d7pq0vy46pp2qv7gh0000gp/T/ipykernel_54918/69679987.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.alpha[j] -= self.learning_rate * error * k_ij
Training Progress: 100%|██████████| 313/313 [16:49<00:00,  3.23s/it]


In [56]:
# モデルの評価
y_pred = classifier.predict(x_test_binary)

# 精度の計算
accuracy = np.mean(y_pred == y_test_binary.flatten())
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.50


In [ ]:
16分かけてaccuracyが0.5とは。そもそもlearning_rateも適切な値がわからないし、RBFカーネルを使ってどうこうという話でもない。
SVMの場合は、双対問題のほうでカーネルトリックが使えるので、陽にk_ij(写像した先での各成分)を計算する必要がない。